In [15]:
AIRPORT = 'KMIA'

In [94]:
import time, os, pathlib, csv, json, subprocess, sys, re
from datetime import datetime

import importlib.util as il
if None in [il.find_spec('FlightRadar24'), il.find_spec('numpy'), \
            il.find_spec('requests'), il.find_spec('pandas'), \
            il.find_spec('matplotlib'), il.find_spec('IPython'), \
            il.find_spec('pyperclip')]:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'FlightRadarAPI']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'numpy']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'requests']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pandas']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'matplotlib']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'IPython']);
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pyperclip']);
os.system('cls')

from FlightRadar24 import FlightRadar24API
import numpy as np
import requests, urllib
import pandas as pd
import matplotlib as mpl, matplotlib.pyplot as plt
from IPython import display
import pyperclip

def pprint(dict):
    print(json.dumps(dict, indent=2))

def pcopy(dict):
    pyperclip.copy(json.dumps(dict, indent=2))

fr_api = FlightRadar24API()

ft_config = fr_api.get_flight_tracker_config()
# ft_config.gnd = '0'
ft_config.vehicles = '0'
fr_api.set_flight_tracker_config(ft_config)

In [103]:
AIRPORT = 'KOPF'
airport_data = fr_api.get_airport_details(AIRPORT)
airport_schedule = airport_data['airport']['pluginData']['schedule']
airport_pos = airport_data['airport']['pluginData']['details']['position']
arrivals = airport_schedule['arrivals']['data']
departures = airport_schedule['departures']['data']

In [86]:
arrs, deps = [], []

for a in departures:
    a = a['flight']
    z = {}

    z['callsign'] = a['identification']['callsign']
    if z['callsign'] is None or a['aircraft'] is None:
        continue
    z['reg'] = a['aircraft']['registration']
    z['type'] = a['aircraft']['model']['code']

    z['origin'] = AIRPORT
    z['dest'] = a['airport']['destination']['code']['icao']
    
    z['gate'] = a['airport']['origin']['info']['gate']
    if z['gate'] is None:
        z['gate'] = 'NA'

    dt = datetime.utcfromtimestamp(a['time']['scheduled']['departure'])
    z['time'] = dt.strftime('%Y-%m-%d %H:%M:%S')

    deps.append(z)

for a in arrivals:
    a = a['flight']
    z = {}

    z['callsign'] = a['identification']['callsign']
    if z['callsign'] is None or a['aircraft'] is None:
        continue
    z['reg'] = a['aircraft']['registration']
    z['type'] = a['aircraft']['model']['code']

    z['origin'] = a['airport']['origin']['code']['icao']
    z['dest'] = AIRPORT 

    z['gate'] = a['airport']['destination']['info']['gate']
    if z['gate'] is None:
        z['gate'] = 'NA'

    dt = datetime.utcfromtimestamp(a['time']['scheduled']['arrival'])
    z['time'] = dt.strftime('%Y-%m-%d %H:%M:%S')

    arrs.append(z)

In [121]:
def get_flight_details(id):
    base_url = 'https://data-live.flightradar24.com/clickhandler/?flight={}'
    data_url = base_url.format(id)
    req = urllib.request.Request(data_url, headers={'User-Agent': 'Magic Browser'})
    
    n_retry = 0
    con = None
    while con is None:
        try:
            con = urllib.request.urlopen(req, timeout=(50 + 10 * n_retry))
        except urllib.request.HTTPError as e:
            if n_retry > 5:
                return None
            n_retry += 1
    return json.loads(con.read().decode('utf8'))

bounds = fr_api.get_bounds_by_point(airport_pos['latitude'], airport_pos['longitude'], 9260)
flights = fr_api.get_flights(bounds = bounds)

for flight in flights:
    z = {}
    z['callsign'] = flight.callsign
    z['id'] = flight.id
    details = get_flight_details(flight.id)
    z['details'] = details
    pcopy(z)

In [122]:
airport_pos

{'latitude': 25.907,
 'longitude': -80.278297,
 'elevation': 8,
 'country': {'name': 'United States', 'code': 'US', 'id': 236},
 'region': {'city': 'Miami'}}